In [1]:
from bs4 import BeautifulSoup
import requests
import time
import pickle
import random
import pandas as ps

In [ ]:
def GetGenreByTitle(artist,title):

    url = "https://www.google.com/search?q="
    headers = {'User-agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.140 Safari/537.36 Edge/17.17134"}
    
    artist = artist.lower()
    artist = artist.replace(" ","+")
    title = title.lower()
    title = title.replace(" ","+")
    
    search = url+title+"+"+artist+"+genre"
    try:
        page = requests.get(search,headers=headers)
        soup = BeautifulSoup(page.content, 'html.parser')
    
        mydivs = soup.find_all("div", {"class":"Z0LcW XcVN5d"})

        if len(mydivs) == 1:
            genre = [str(mydivs)[27:-7]]
        else:
            mydivs = soup.find_all("div", {"class":"rlc__slider-page"})
            genre = []
            for element in range(len(mydivs)):
                genre_title = mydivs[element].find("div", {"class":"title"})
                genre_title = str(genre_title)[19:-6]
                genre.append(genre_title)
    except:
        genre = "Search for Genre failed"
    time.sleep(1)
    return genre

In [ ]:
tracks_df = ps.DataFrame(columns = ["Artist","Song_title","Song_text","Song_genre"])

# Website

In [2]:
website = "https://www.songtexte.de/musiker.html?seite="

In [3]:
weblist = []
for x in range(1,51):
    weblist.append(website+str(x))

In [4]:
# Dict bestehend aus {Seitenzahl:{Künstler:[link],...}...}

In [5]:
songdict = {}
counter = 1
for url in weblist:
    songdict.update({counter:{}})
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    mydivs = soup.find_all("div", {"class": "cover"})
    for c in range(len(mydivs)):
        tag = mydivs[c].a.attrs
        link = tag["href"]
        artist = tag["title"][:-34]
        songdict[counter].update({artist:link})
    counter += 1
    time.sleep(random.random()*1+1)

KeyboardInterrupt: 

In [ ]:
# Dict bestehend aus {{Künstler:{Liedname:[Liedurl]},...}...}

In [ ]:
artistdict = {}
counter = 0
fail = {}

for element in range(1,len(songdict.keys())+1): #Für jede Seite
    for artist in songdict[element].keys(): #Für jeden Künstler
        counter += 1
        songs = {}
        url = songdict[element][artist]
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        try:
            mydivs = soup.find_all("div", {"class": "album-content col-sm-9"})[0]
            mydivs = mydivs.find("ul")
            tags = mydivs.find_all("a")
            #print(len(tags))
            for c in range(len(tags)): #Für jeden Song des Künstlers
                song = tags[c]["title"][len(artist)+3:-9]
                link = tags[c]["href"]
                songs.update({song:[link]})
                artistdict.update({artist:songs})
            #print(counter, artist, "erfolgreich")
        except:
            fail.update({artist:mydivs})
            print(counter, artist, "fehlgeschlagen Link:", songdict[element][artist])

        counter += 1
        time.sleep(random.random()*1+1)

In [ ]:
# Dict bestehend aus {{Künstler:{Liedname:[Liedurl,Lyrics],...}...}

In [ ]:
counter = 0
for artist in artistdict.keys():
    for song in artistdict[artist]:
        try:
            url = artistdict[artist][song][0]
            page = requests.get(url)
            soup = BeautifulSoup(page.content, 'html.parser')
            mydivs = soup.find_all("div", {"class": "album-content col-sm-9 col-xs-12"})
            tag = mydivs[0].find_all("p")[1]
            song_text = str(tag)[23:-22].replace("<br>","").replace("<br/>","")
            #artistdict[artist][song].append(song_text)
            genre = GetGenreByTitle(artist,song)
            tracks_df[counter] = [artist,song,song_text,genre]             
            counter +=1
            time.sleep(random.random()*1+0.5)
        except:
            print("Fehler bei", artist, song, url)
        if counter%500 == 0:
            print(counter,"Lieder heruntergeladen")
        

In [ ]:
#artistdict["Ed Sheeran"]['Thinking Out Loud'][0]

In [ ]:
# counter = 0
# for artist in artistdict.keys():
#     for song in artistdict[artist]:
#         if len(artistdict[artist][song]) == 2:
#             genre = GetGenreByTitle(artist,song)
#             tracks_df[counter] = [artist,song,artistdict[artist][song][1],genre]
#             counter +=1

In [ ]:
tracks_df.head()

In [ ]:
output = open('tracks50.pkl', 'wb')
pickle.dump(tracks_df, output)
output.close()